<p><font size="7" color='grey'> <b>
Anwendung Generativer KI
</b></font> </br></p>

<p><font size="6" color='grey'> <b>
Modul 07: Retrieval Augmented Generation
</b></font> </br></p>


---

# 1 | Übersicht
---

# 6.1: Einführung in Retrieval-Augmented Generation (RAG)

Large Language Models (LLMs), wie sie in LangChain integriert sind, sind leistungsstarke Tools zum Verarbeiten und Verstehen großer Textmengen. Diese Modelle können genutzt werden, um Fragen basierend auf dem Inhalt eines Dokuments zu beantworten, was sie für Aufgaben, die Informationsextraktion und -verständnis erfordern, äußerst wertvoll macht.

Eine der Techniken, die in Verbindung mit LLMs zur dokumentenbasierten Beantwortung von Fragen verwendet werden, ist die Retrieval-Augmented Generation (RAG). RAG ist ein hybrider Ansatz, der die Stärken von Informationsabrufsystemen mit den generativen Fähigkeiten von Sprachmodellen kombiniert. Hier ist ein kurzer Überblick über die Funktionsweise von RAG:

1. **Abrufphase:** Wenn eine Frage gestellt wird, ruft das RAG-System zunächst relevante Dokumente oder Dokumentsegmente aus einem großen Korpus ab. Dieser Abruf basiert auf der Ähnlichkeit des Inhalts der Dokumente mit der Frage. Die Idee besteht darin, Textbeweise zu finden, die die Antwort enthalten könnten.
2. **Augmentationsphase:** Die abgerufenen Dokumente werden dann als Kontextinformationen in ein Sprachmodell eingespeist. Dieser Schritt ist entscheidend, da er dem Sprachmodell spezifische, für die Frage relevante Daten liefert, die in der vorab trainierten Wissensbasis des Modells möglicherweise nicht vorhanden sind.
3. **Generierungsphase:** Mit dem Kontext der abgerufenen Dokumente generiert das Sprachmodell eine Antwort. Das Modell synthetisiert die Informationen aus den Dokumenten und verwendet sein Sprach- und Kontextverständnis, um eine kohärente und genaue Antwort zu formulieren.

Durch die Kombination des Abrufs relevanter Informationen mit der generativen Leistung von LLMs verbessert RAG effektiv die Fähigkeit des Modells, präzise Antworten basierend auf dem Inhalt eines Dokuments zu liefern. Dieser Ansatz ist besonders nützlich in Szenarien, in denen die direkte Antwort auf eine Frage möglicherweise nicht ohne weiteres in den Trainingsdaten des Modells verfügbar ist und das System externe Beweise abrufen muss, um die Antwortgenerierung zu unterstützen. Dies macht die Integration von LLMs mit RAG durch LangChain zu einer robusten Lösung für die dokumentenbasierte Beantwortung von Fragen, die ein tiefes Verständnis und differenzierte Antworten in verschiedenen Bereichen und Arten von Anfragen ermöglicht.

Wir beginnen mit dem Öffnen einer Verbindung zu einem OpenAI LLM-Modell.

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain import OpenAI, PromptTemplate
from langchain_openai import ChatOpenAI
from IPython.display import display_markdown

MODEL = 'gpt-4o-mini'

llm = ChatOpenAI(
        model=MODEL,
        temperature=0.2,
        n=1
    )


Wir werden nun mehrere PDFs laden, die wir mit Fragen abfragen können. Wir können zu diesen Dokumenten Fragen mit Informationen stellen, die nicht Teil des Basismodells sind. Wir erstellen Loader für jedes der PDFs, damit wir sie zur einfachen Abfrage in einen Vektorspeicher laden können.

Der folgende Codeausschnitt zeigt, wie man mit einer bestimmten Funktion „load_summarize_chain“ einen Zusammenfassungsprozess mithilfe eines Large Language Model (LLM) mit einem Kettentyp „map_reduce“ einrichtet. Zunächst wird mithilfe des „PyPDFLoader“ ein PDF von der angegebenen URL („https://arxiv.org/pdf/1706.03762“) geladen. Das geladene Dokument wird dann in handhabbare Teile aufgeteilt („load_and_split“). Diese Teile werden in die Zusammenfassungskette („chain.run(docs)“) eingespeist, die den Inhalt verarbeitet und verdichtet. Schließlich wird der zusammengefasste Inhalt im Markdown-Format direkt in der Ausgabeumgebung angezeigt, wodurch sichergestellt wird, dass die Formatierung der Zusammenfassung erhalten bleibt.

In [ ]:
urls = [
  "https://arxiv.org/pdf/1706.03762",
  "https://arxiv.org/pdf/1810.04805",
  "https://arxiv.org/pdf/2005.14165",
  "https://arxiv.org/pdf/1910.10683"
]

loaders = []

chain = load_summarize_chain(llm, chain_type="map_reduce")

for url in urls:
print(f"Lesen: {url}")
  loader = PyPDFLoader(url)
  loaders.append(loader)



Reading: https://arxiv.org/pdf/1706.03762
Reading: https://arxiv.org/pdf/1810.04805
Reading: https://arxiv.org/pdf/2005.14165
Reading: https://arxiv.org/pdf/1910.10683


Als nächstes laden wir Einbettungen aus den vier Dokumenten in [ChromaDB](https://www.trychroma.com/). Diese Einbettungen ermöglichen es, die Eingabeaufforderung mit Informationen aus den von uns geladenen PDF-Dokumenten zu erweitern.

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.inmemory import InMemoryVectorStore

embeddings_model = OpenAIEmbeddings()
index = VectorstoreIndexCreator(embedding=embeddings_model,vectorstore_cls=InMemoryVectorStore).from_loaders(loaders)

Nachdem die Einbettungen geladen sind, können wir das Modell nach Informationen abfragen, die nur in den Dokumenten enthalten sind.

In [ ]:
query = "Which figure demonstrates Scaled Dot-Product Attention?"

index.query(query,llm=llm)

'The left figure in Figure 2 demonstrates Scaled Dot-Product Attention.'

## Einschränkungen von RAG

Language Model Retrieval-Augmented Generation (LLM RAG) kombiniert die Fähigkeiten großer Sprachmodelle mit externen Datenabrufmechanismen. Dieser Ansatz verbessert die Leistung von Sprachmodellen, indem er Zugriff auf spezifische, oft proprietäre Daten bietet, die dem allgemeinen Wissen des vorab trainierten Modells möglicherweise fehlen. Die Wirksamkeit von LLM RAG nimmt jedoch erheblich ab, wenn die erweiterten Daten bereits Allgemeinwissen sind und inhärent im Basismodell enthalten sind.

LLM RAG eignet sich hervorragend für Szenarien, in denen Benutzer proprietäre oder hochspezialisierte Informationen benötigen. In Bereichen wie Finanzen, Recht oder technischen Branchen sind spezifische Datensätze, Berichte oder Dokumente für genaue Antworten unerlässlich. Die einzigartige Fähigkeit von RAG, diese externen Daten zu nutzen, gewährleistet hochpräzise und kontextbezogen relevante Antworten. Obwohl das Basismodell umfassend anhand einer breiten Palette öffentlich verfügbarer Informationen trainiert wurde, fehlt ihm möglicherweise die Tiefe oder die neuesten Updates, die für diese Nischenbereiche erforderlich sind. Daher führt die Integration proprietärer Datensätze durch RAG zu genaueren und kontextbezogen angereicherten Ergebnissen.

Es ist wichtig zu bedenken, dass sich die Vorteile von LLM RAG erheblich verringern, wenn die zu erweiternden Daten bereits Allgemeinwissen sind. Das Basismodell wird vorab anhand großer Datenmengen trainiert, die ein breites Spektrum allgemeiner Wissensthemen abdecken. Wenn Abfragen Informationen beinhalten, die in diesen allgemeinen Rahmen fallen, generiert das Basismodell daher genaue und informative Antworten, ohne dass eine externe Erweiterung erforderlich ist. In solchen Fällen bietet die Verwendung von RAG keinen nennenswerten Mehrwert und kann zu unnötiger Komplexität und Verarbeitungsaufwand führen.

Darüber hinaus führt das Abrufen von Daten, die das Modell bereits gut versteht, zu Ineffizienzen. Das umfangreiche Vortraining des Basismodells umfasst unterschiedliche Texte, was bedeutet, dass seine interne Wissensbasis normalerweise für allgemeine Wissensabfragen ausreicht. Wenn man sich in diesen Fällen auf RAG verlässt, werden die Stärken des Modells daher nicht effektiv genutzt und die Gesamteffizienz des Systems beeinträchtigt.

Zusammenfassend lässt sich sagen, dass LLM RAG zwar sehr hilfreich ist, wenn es um die Erweiterung von Sprachmodellen mit proprietären oder hochspezialisierten Daten geht, seine Wirksamkeit jedoch nachlässt, wenn es um allgemeines Wissen geht. Das umfassende Training des Basismodells deckt bereits eine große Menge allgemeiner Informationen ab, sodass eine RAG-Erweiterung in solchen Kontexten überflüssig wird. Setzen Sie RAG daher strategisch ein, da es die größte Verbesserung in Bereichen bietet, in denen Zugriff auf proprietäre Daten erforderlich ist, die das Basismodell möglicherweise nur teilweise abdeckt.

# Modul 6 Aufgabe

Die erste Aufgabe findet ihr hier: [assignment 6](https://github.com/jeffheaton/app_generative_ai/blob/main/assignments/assignment_yourname_t81_559_class6.ipynb)

# 6.2: Einführung in ChromaDB

In diesem Modul untersuchen wir die grundlegende Verwendung von Chroma, um Informationen effizient durch Einbettungen zu speichern und abzurufen. Diese grundlegende Technik bildet das Rückgrat zahlreicher fortschrittlicher KI-Anwendungen.

* [ChromaDB](https://www.trychroma.com/)

Einbettungen dienen als KI-native Darstellung verschiedener Datentypen und sind daher ideal für die Verwendung mit einer breiten Palette KI-gestützter Tools und Algorithmen. Sie können die Essenz von Text, Bildern und bald sogar Audio und Video zusammenfassen.

Ein Einbettungsmodell verarbeitet Daten, um eine Einbettung zu erzeugen, die Vektorzahlen erzeugt. Die Entwickler haben das Modell so konzipiert, dass ähnliche Daten, wie etwa Texte mit analogen Bedeutungen oder Bilder mit vergleichbarem Inhalt, Vektoren ergeben, die im Vektorraum näher beieinander liegen, während unähnliche Daten zu weiter entfernten Vektoren führen.

Die Architektur von ChromaDB ist in Abbildung 6.ChromaDB dargestellt.

**Abbildung 6.ChromaDB: Die Architektur von ChromaDB**
![ChromaDB](https://data.heatonresearch.com/images/wustl/app_genai/hrm4.svg)

Die Kern-API von ChromaDB besteht aus nur vier API-Aufrufen, von denen der erste den Client einrichtet, den Sie für die Interaktion mit ChromaDB verwenden.

```
import chromadb
client = chromadb.HttpClient()
```

Mit dem Client können Sie eine Sammlung erstellen, die Dokumente enthält.

```
collection = client.create_collection("sample_collection")
```

Als nächstes fügen wir dieser Sammlung Dokumente hinzu.

```
collection.add(
 documents=["This is document1", "This is document2"], # we embed for you, or bring your own
 metadatas=[{"source": "notion"}, {"source": "google-docs"}], # filter on arbitrary metadata!
 ids=["doc1", "doc2"], # must be unique for each doc
)
```

Wenn alles vorhanden ist, können Sie jetzt eine Abfrage durchführen.

```
results = collection.query(
 query_texts=["This is a query document"],
 n_results=2,
 # where={"metadata_field": "is_equal_to_this"}, # optional filter
 # where_document={"$contains":"search_string"}  # optional filter
)
```

## Laden und Abfragen von ChromaDB

Wir sehen uns nun ein Beispiel für das Laden und Abfragen von ChromeDB an. Dazu verwenden wir den [SciQ dataset](https://huggingface.co/datasets/allenai/sciq) von HuggingFace. Der SciQ-Datensatz enthält 13.679 Crowdsourcing-Prüfungsfragen zu Naturwissenschaften, unter anderem zu Physik, Chemie und Biologie. Die Fragen sind im Multiple-Choice-Format mit jeweils 4 Antwortoptionen. Zusätzlich haben die Autoren einen Absatz mit Belegen für die richtige Antwort auf die meisten Fragen bereitgestellt.

Wir beginnen damit, diese Fragen mit unterstützenden Informationen anzureichern.



In [ ]:
# Holen Sie sich den SciQ-Datensatz von HuggingFace
from datasets import load_dataset

dataset = load_dataset("sciq", split="train")

# Filtern Sie den Datensatz, um nur Fragen mit einer Unterstützung einzuschließen
dataset = dataset.filter(lambda x: x["support"] != "")

print("Number of questions with support: ", len(dataset))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11679 [00:00<?, ? examples/s]

Number of questions with support:  10481


Wir beginnen mit der Erstellung eines ChromaDB-Clients und einer Sammlung namens „sciq_supports“. Der Standard-Chroma-Client ist flüchtig, d. h. er speichert nicht auf der Festplatte. In Teil 5 erfahren Sie, wie diese Daten gespeichert werden. Wir erstellen eine neue Chroma-Sammlung, um die unterstützenden Beweise zu speichern. Wir müssen keine Einbettungsfunktion angeben; wir verwenden die Standardfunktion.

In [ ]:
import chromadb

client = chromadb.Client()

collection = client.create_collection("sciq_supports")

Als nächstes betten wir die ersten 100 Stützen für dieses Beispiel ein und speichern sie.

In [ ]:
collection.add(
    ids=[str(i) for i in range(0, 100)],  # IDs sind nur Zeichenfolgen
    documents=dataset["support"][:100],
    metadatas=[{"type": "support"} for _ in range(0, 100)
    ],
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:17<00:00, 4.87MiB/s]


Wir fragen nun die Datenbank nach Unterstützung für jede der Fragen ab.

In [ ]:
results = collection.query(
    query_texts=dataset["question"][:10],
    n_results=1)

Wir lassen uns diese Informationen anzeigen und können sehen, welche Teile des Dokuments die einzelnen Fragen unterstützen. Später werden wir sehen, dass RAG diese Unterstützung in die Eingabeaufforderung zur Beantwortung der Frage einspeist.

In [ ]:
# Drucken Sie die Frage und die dazugehörige Hilfestellung aus
for i, q in enumerate(dataset['question'][:10]):
print(f"Frage: {q}")
print(f"Abgerufener Support: {results['documents'][i][0]}")
    print()

Question: What type of organism is commonly used in preparation of foods such as cheese and yogurt?
Retrieved support: Agents of Decomposition The fungus-like protist saprobes are specialized to absorb nutrients from nonliving organic matter, such as dead organisms or their wastes. For instance, many types of oomycetes grow on dead animals or algae. Saprobic protists have the essential function of returning inorganic nutrients to the soil and water. This process allows for new plant growth, which in turn generates sustenance for other organisms along the food chain. Indeed, without saprobe species, such as protists, fungi, and bacteria, life would cease to exist as all organic carbon became “tied up” in dead organisms.

Question: What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?
Retrieved support: Without Coriolis Effect the global winds would blow north to south

# 6.3: Einbettungen verstehen

Ein [embedding](https://platform.openai.com/docs/guides/embeddings) ist ein Vektor (eine Liste) von Gleitkommazahlen. Der [embedding](https://platform.openai.com/docs/guides/embeddings) zwischen zwei Vektoren misst deren Verwandtschaftsgrad. Kleine Abstände deuten auf einen hohen Verwandtschaftsgrad hin, große Abstände auf einen niedrigen Verwandtschaftsgrad.

Die beiden Einbettungsmodelle, zwischen denen Sie in OpenAI wählen können, sind die folgenden:

* Text-Einbettung-3-klein
* Text-Einbettung-3-groß

Die Wahl zwischen den beiden OpenAI-Einbettungsmodellen „text-embedding-3-small“ und „text-embedding-3-large“ hängt von mehreren Faktoren ab, die mit Ihrem spezifischen Anwendungsfall zusammenhängen, darunter Leistungsanforderungen, Rechenressourcen und die Art der Aufgaben, für die Sie die Einbettungen benötigen. Hier sind einige wichtige Überlegungen:

1. **Leistung und Genauigkeit**:
* **text-embedding-3-large**: Im Allgemeinen erfassen größere Modelle differenziertere und komplexere Beziehungen innerhalb des Textes. Dies führt zu einer besseren Leistung bei Aufgaben, die ein tiefes Verständnis der Sprache erfordern, wie z. B. semantische Ähnlichkeit, Stimmungsanalyse und anspruchsvollere NLP-Aufgaben.
* **text-embedding-3-small**: Kleinere Modelle sind möglicherweise nicht so genau oder detailliert wie größere, sie können aber dennoch viele Aufgaben gut bewältigen, insbesondere solche mit geringerer Komplexität oder bei Feinabstimmung auf bestimmte Datensätze.
2. **Rechenressourcen:**
* **text-embedding-3-large**: Erfordert mehr Rechenleistung und Speicher. Diese Anforderung ist wichtig, wenn Sie das Modell in einer ressourcenbeschränkten Umgebung einsetzen oder eine große Datenmenge in Echtzeit verarbeiten müssen.
* **text-embedding-3-small**: Effizientere Ressourcennutzung und daher die bessere Wahl für Anwendungen mit begrenzter Rechenleistung oder bei einem Betrieb in einem Maßstab, bei dem Kosten und Geschwindigkeit entscheidende Faktoren sind.
3. **Latenz und Durchsatz:**
* **text-embedding-3-large**: Größere Modelle weisen aufgrund ihrer Komplexität normalerweise eine höhere Latenz auf, was sich auf Echtzeitanwendungen auswirken kann.
* **text-embedding-3-small**: Geringere Latenz und schnellere Inferenzzeiten kommen Anwendungen zugute, die schnelle Reaktionen erfordern.
4. **Kosten:**
* **text-embedding-3-large**: Aufgrund des höheren Rechenleistungsbedarfs sind wahrscheinlich höhere Betriebskosten verbunden.
* **text-embedding-3-small**: Kostengünstiger, insbesondere bei groß angelegten Bereitstellungen.
Anwendungsfallspezifikationen:

Das große Modell ist möglicherweise besser für Anwendungen geeignet, die eine hohe Präzision erfordern und bei denen ein detailliertes semantisches Verständnis von entscheidender Bedeutung ist, wie etwa bei der differenzierten Textanalyse oder der fortgeschrittenen KI-Forschung.
Das kleine Modell könnte die bessere Wahl für Anwendungen sein, bei denen Geschwindigkeit, Kosten und Effizienz wichtiger sind, wie etwa Echtzeitsysteme, Chatbots oder Anwendungen mit einfacheren Textverarbeitungsanforderungen.

Zusammenfassend:

* Wählen Sie text-embedding-3-large, wenn:
* Sie benötigen hohe Genauigkeit und ein detailliertes semantisches Verständnis.
* Sie verfügen über ausreichend Rechenressourcen und ein ausreichendes Budget.
* Die Latenz ist kein kritisches Problem.
* Wählen Sie text-embedding-3-small, wenn:
* Sie legen Wert auf eine effiziente Ressourcennutzung und geringere Kosten.
* Sie benötigen schnellere Inferenzzeiten.

Die Aufgaben sind weniger komplex oder die Umgebung erfordert eingeschränktere Ressourcen.
Durch die Bewertung Ihrer Anforderungen und Einschränkungen können Sie leichter entscheiden, welches Modell Sie verwenden möchten.

## Instanziieren eines Einbettungsmodells

Für diesen Kurs empfehle ich die Verwendung von **text-embedding-3-small**. Es verfügt über alle Funktionen, die wir benötigen, und wird Ihre Credits noch weiter aufwerten. Beginnen wir mit der Erstellung eines Clients, der dieses Modell verwendet.


In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

Beginnen wir mit dem Verständnis der Grundlagen eines Einbettungsmodells und von Vektoren. Ein Einbettungsmodell ist ein Tool, das jede Textzeichenfolge, unabhängig von ihrer Länge, in einen Vektor umwandeln kann. Dieser Vektor ist eine eindeutige Darstellung der Textzeichenfolge. Wenn die Vektoren für zwei Textzeichenfolgen gleich sind, bedeutet dies, dass die Textzeichenfolgen identisch sind. Wenn die Vektoren unterschiedlich sind, deutet dies darauf hin, dass die Textzeichenfolgen unterschiedlich sind. Dieser Unterschied ist jedoch kein einfacher Vergleich. Sogar zwei sehr unterschiedliche Textzeichenfolgen mit derselben Bedeutung können ähnliche Vektoren erzeugen.

Sehen wir uns zunächst Vektoren für einzelne Wörter an.

In [ ]:
l1 = embeddings_model.embed_query("dog")
l2 = embeddings_model.embed_query("Something that is a bit longer than a word.")

print(type(l1))

<class 'list'>


Wie Sie sehen, ist die Ausgabe nur eine normale Python-Liste. Diese Listen sind ziemlich lang.

In [ ]:
print(len(l1))
print(len(l2))

1536
1536


Die Länge dieser Zeichenfolge bleibt bei allen Abfragen desselben Modells gleich. Die größere Version des Modells kann zwar besser Vektoren erstellen, die Zeichenfolgen unterscheiden, erfordert aber nicht immer eine größere Vektorlänge, um diese Qualitätssteigerung zu erreichen. Es handelt sich um ein differenziertes Konzept, das einer weiteren Untersuchung bedarf.

Wenn wir die eigentliche Liste anzeigen, sehen wir, dass es sich lediglich um eine Sammlung von Zahlen handelt. Hier zeigen wir nur die ersten zehn Elemente an.

In [ ]:
print(l1[:10])

[0.05113774910569191, -0.01870863139629364, -0.004298428073525429, 0.07271610200405121, -0.007174310740083456, -0.014693480916321278, -0.0059395902790129185, 0.005037412978708744, 0.018954960629343987, -0.01090618409216404]


## Vektoren vergleichen

Um diese Vektoren zu vergleichen, verwenden wir die mathematischen Fähigkeiten von [Numpy](https://numpy.org/). Es gibt viele verschiedene Ansätze, um Vektoren in der Mathematik zu vergleichen. Einige der gängigsten werden hier vorgestellt.

* **Skalarprodukt**: Misst den Kosinus des Winkels zwischen zwei Vektoren und gibt so ihre Richtungsähnlichkeit an. Wird zur Bestimmung der Orthogonalität, Projektion und in verschiedenen Anwendungen wie Computergrafik und maschinellem Lernen verwendet.

* **Kreuzprodukt**: Berechnet einen Vektor senkrecht zu zwei gegebenen Vektoren im dreidimensionalen Raum, nützlich zum Ermitteln des Normalvektors einer Ebene und in der Physik für Drehmoment- und Drehimpulsberechnungen.

* **Euklidische Distanz**: Berechnet die geradlinige Distanz zwischen zwei Vektoren. Wird häufig im maschinellen Lernen für Clustering- und Nächster-Nachbar-Algorithmen verwendet.

* **Kosinus-Ähnlichkeit**: Bewertet den Kosinus des Winkels zwischen zwei Vektoren und betont dabei die Ausrichtung statt der Größe. Wird häufig beim Text Mining und Informationsabruf verwendet, um die Ähnlichkeit von Dokumenten zu vergleichen.

* **Manhattan-Distanz**: Misst die Summe der absoluten Differenzen zwischen den Komponenten zweier Vektoren, nützlich in gitterbasierten Pfadfindungsalgorithmen und einigen Anwendungen des maschinellen Lernens.

OpenAI [suggests](https://platform.openai.com/docs/guides/embeddings/frequently-asked-questions), dass wir Cosinus-Ähnlichkeit verwenden, um ihre Vektoren zu vergleichen, da die Größe erhalten bleibt. Das Vorzeichen der einzelnen Vektornummern ist wichtig, wir möchten es nicht verwerfen.

Wir beginnen mit der Konvertierung der beiden Einbettungen von Python-Listen in Numpy-Arrays.

In [ ]:
import numpy as np

# Konvertieren Sie Listen in Numpy-Arrays
vec1 = np.array(l1)
vec2 = np.array(l2)

OpenAI gibt an, dass alle Einbettungen auf die Länge 1 normalisiert sind, was häufig [unit vectors](https://en.wikipedia.org/wiki/Unit_vector) genannt wird. Dies bedeutet, dass:

* Die Kosinus-Ähnlichkeit kann etwas schneller berechnet werden, wenn man nur ein Skalarprodukt verwendet
* Kosinusähnlichkeit und euklidische Distanz führen zu identischen Rankings

Lassen Sie uns dies nun in die Praxis umsetzen. Um zu überprüfen, ob diese Einbettungen tatsächlich die Länge 1 haben, können wir eine praktische Funktion verwenden. Diese Funktion ist speziell für die Analyse der Länge eines Vektors konzipiert und ist daher ein nützliches Werkzeug bei unserer Untersuchung von OpenAI-Einbettungen.

In [ ]:
def analyze_vector_length(vector):
  # Berechnen Sie die Länge (Norm) des Vektors
  length = np.linalg.norm(vector)

  # Überprüfen Sie, ob der Vektor ein Einheitsvektor ist
  is_unit_vector = np.isclose(length, 1.0)

print(f"Vektor: {Vektor}")
print(f"Länge des Vektors: {length}")
print(f"Ist der Vektor ein Einheitsvektor? {'Ja' wenn is_unit_vector sonst 'Nein'}")

  # Normalisieren Sie den Vektor, um ihn zu einem Einheitsvektor zu machen
  unit_vector = vector / length

  # Überprüfen Sie die Länge des normalisierten Vektors
  normalized_length = np.linalg.norm(unit_vector)

print(f"Normalisierter Vektor (Einheitsvektor): {unit_vector}")
print(f"Länge des normalisierten Vektors: {normalized_length}")

In [ ]:
analyze_vector_length(vec1)

Vector: [ 0.05113775 -0.01870863 -0.00429843 ...  0.02879578  0.00215999
  0.01790806]
Length of the vector: 0.9999999997385218
Is the vector a unit vector? Yes
Normalized vector (unit vector): [ 0.05113775 -0.01870863 -0.00429843 ...  0.02879578  0.00215999
  0.01790806]
Length of the normalized vector: 1.0000000000000002


Als nächstes berechnen wir die tatsächliche Kosinus-Ähnlichkeit zwischen unseren beiden Vektoren.

In [ ]:
import numpy as np

# Konvertieren Sie Listen in Numpy-Arrays
vec1 = np.array(l1)
vec2 = np.array(l2)

# Berechnen Sie das Skalarprodukt
dot_product = np.dot(vec1, vec2)

# Berechnen Sie die Größen (L2-Normen)
magnitude_vec1 = np.linalg.norm(vec1)
magnitude_vec2 = np.linalg.norm(vec2)

# Kosinus-Ähnlichkeit berechnen
cosine_similarity = dot_product / (magnitude_vec1 * magnitude_vec2)

print(f"Kosinus-Ähnlichkeit: {cosine_similarity}")

Cosine similarity: 0.16737720056743507


Der Nenner oben ist 1,0, wie hier gezeigt. Dies liegt an der Einheitsvektoreigenschaft. Wir können unseren Einheitsvektorvergleich auf das Skalarprodukt vereinfachen, wie von OpenAI angegeben.

In [ ]:
print(magnitude_vec1 * magnitude_vec2)

1.0000000478179367


Das Skalarprodukt kann wie folgt berechnet werden.

In [ ]:
print(np.dot(vec1, vec2))

0.16737720857106747


## Ähnlichkeiten von Zeichenfolgen auswerten

Wir beginnen mit der Erstellung einer einfachen Funktion zum Vergleichen zweier Zeichenfolgen.

In [ ]:
def compare_str(embeddings_model, text1, text2):
    """
    This function returns the dot product of embeddings for two given text strings.

    Parameters:
    embeddings_model: The embeddings model to use for generating embeddings.
    text1 (str): The first text string.
    text2 (str): The second text string.

    Returns:
    float: The dot product of the embeddings for text1 and text2.
    """
    # Holen Sie sich die Einbettungen für die beiden Textzeichenfolgen
    embedding1 = embeddings_model.embed_query(text1)
    embedding2 = embeddings_model.embed_query(text2)

    # Konvertieren Sie Einbettungen in Numpy-Arrays zur Skalarproduktberechnung
    embedding1_array = np.array(embedding1)
    embedding2_array = np.array(embedding2)

    # Berechnen und Zurückgeben des Skalarprodukts
    dot_product = np.dot(embedding1_array, embedding2_array)
    return dot_product

Versuchen wir es mit zwei Beschreibungen eines „Rasenmähers“, in denen nicht viele ähnliche Wörter verwendet werden.

In [ ]:
compare_str(embeddings_model,
            "A machine that helps people to cut grass.",
            "Device with blades to cut plants under it.")

0.6298291212454291

Der Wert des Skalarprodukts (Kosinusähnlichkeit) reicht von -1 bis 1. Wir können ihn wie folgt interpretieren:

* Hohe Ähnlichkeit: Werte nahe 1 weisen auf eine hohe Ähnlichkeit hin.
* Geringe Ähnlichkeit: Werte nahe -1 weisen auf eine hohe Unähnlichkeit hin.
* Neutral/Keine Ähnlichkeit: Werte nahe 0 zeigen keine offensichtliche Ähnlichkeit an.

Ein Wert von 0,62 bedeutet also, dass sie einigermaßen ähnlich sind. Passen wir ihn an, um einen Rasenmäher mit einem Flugzeug zu vergleichen.

In [ ]:
compare_str(embeddings_model,
            "A machine that helps people to cut grass.",
            "Vehicle that flys through the air.")

0.2694946993627674

Wir können sehen, dass die Ähnlichkeit geringer ist.

# 6.4: Beantwortung von Fragen über Textdokumente

Retrieval-Augmented Generation (RAG) ist eine fortschrittliche Technik, die die Fähigkeiten großer Sprachmodelle (LLMs) durch die Integration externer Daten in den Antwortgenerierungsprozess verbessern soll. Damit RAG wirklich effektiv ist, muss es mit Daten arbeiten, die noch nicht im Basismodell vorhanden sind. Dies ist entscheidend, da der Hauptvorteil von RAG in seiner Fähigkeit liegt, spezifische, oft aktuelle Informationen abzurufen, mit denen das LLM während seines anfänglichen Trainings möglicherweise nicht in Berührung gekommen ist.

Diese Funktion macht RAG besonders wertvoll für Unternehmensumgebungen. Unternehmen generieren und speichern große Mengen an proprietären Daten, darunter interne Dokumente, Kundeninformationen und detaillierte Berichte. Diese Daten sind oft unternehmensspezifisch und nicht Teil des öffentlich verfügbaren Korpus, mit dem ein LLM geschult würde. Durch die Verwendung von RAG können Unternehmen ihre eigenen Daten nutzen, um präzisere und kontextrelevantere Antworten von ihren LLMs zu erhalten und so Entscheidungsprozesse und Betriebseffizienz zu verbessern.

Um die Fähigkeiten von RAG zu veranschaulichen, verwenden wir einen von mir erstellten Beispieldatensatz, der synthetische Daten von Mitarbeiterbiografien aus fünf fiktiven Unternehmen enthält. Dieser Datensatz soll zeigen, wie RAG bestimmte Informationen, die ein Basismodell nicht von sich aus besitzen würde, effektiv abrufen und nutzen kann. Auf diese Weise liefert er ein klares Beispiel dafür, wie RAG in einem realen Unternehmensumfeld angewendet werden kann.

Hier ist ein Beispiel für eine solche generierte Biografie:

> Elena Martinez ist eine erfahrene Robotikingenieurin bei FutureTech, einem führenden Innovator im Bereich künstliche Intelligenz und Robotik mit Sitz im Silicon Valley. Mit einem Master-Abschluss in Maschinenbau vom MIT und über einem Jahrzehnt Erfahrung war Elena maßgeblich an der Entwicklung autonomer Robotersysteme beteiligt, die die städtische Mobilität und Zugänglichkeit verbessern sollen. Zu ihren bahnbrechenden Arbeiten gehört die Entwicklung des ersten KI-gesteuerten Roboterassistenten, der nahtlos mit städtischen Umgebungen interagieren kann, um älteren und behinderten Menschen zu helfen. Als leidenschaftliche Verfechterin der Förderung von Frauen in MINT-Fächern leitet Elena auch das Outreach-Programm von FutureTech, das darauf abzielt, die nächste Generation weiblicher Ingenieure durch Workshops und Mentoring zu inspirieren. Ihre Beiträge haben FutureTech nicht nur zu neuen Höhen geführt, sondern auch neue Maßstäbe bei Roboteranwendungen für das Gemeinwohl gesetzt.


Diese Biografie zeigt die Art detaillierter, unternehmensspezifischer Informationen, die RAG abrufen und in seine Antworten integrieren kann, wodurch die Relevanz und Genauigkeit der generierten Inhalte verbessert wird. Durch die Integration dieser maßgeschneiderten Daten bereichert RAG nicht nur die Ergebnisse der LLMs, sondern stellt auch sicher, dass die Antworten auf den einzigartigen Kontext und die Bedürfnisse der Organisation abgestimmt sind.

Diese Beispieldaten werden unter den folgenden URLs gespeichert. Jede Datei enthält Personen aus einem der Unternehmen.

* https://data.heatonresearch.com/data/t81-559/bios/DD.txt
* https://data.heatonresearch.com/data/t81-559/bios/FT.txt
* https://data.heatonresearch.com/data/t81-559/bios/GS.txt
* https://data.heatonresearch.com/data/t81-559/bios/NGS.txt
* https://data.heatonresearch.com/data/t81-559/bios/TI.txt

Der folgende Code definiert diese URLs und instanziiert ein LLM-Modell.



In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import TextLoader
from langchain import OpenAI, PromptTemplate
from langchain_openai import ChatOpenAI
from IPython.display import display_markdown
from langchain.indexes import VectorstoreIndexCreator
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.inmemory import InMemoryVectorStore
from langchain.schema import Document
import requests

MODEL = 'gpt-4o-mini'

llm = ChatOpenAI(
        model=MODEL,
        temperature=0.2,
        n=1
    )

urls = [
    "https://data.heatonresearch.com/data/t81-559/bios/DD.txt",
    "https://data.heatonresearch.com/data/t81-559/bios/FT.txt",
    "https://data.heatonresearch.com/data/t81-559/bios/GS.txt",
    "https://data.heatonresearch.com/data/t81-559/bios/NGS.txt",
    "https://data.heatonresearch.com/data/t81-559/bios/TI.txt"
]


Die Funktion verarbeitet ein großes Dokument, indem sie es in kleinere, handhabbare Segmente, sogenannte Chunks, unterteilt, um Einbettungen für den LLM-Abruf in Retrieval-Augmented Generation (RAG) zu erstellen. Der Chunk-Parameter bestimmt die maximale Länge jedes Segments und stellt sicher, dass der Text in Teile zerlegt wird, die vom Sprachmodell effizient verarbeitet und analysiert werden können. Der Overlap-Parameter gibt die Anzahl der Token an, die am Anfang jedes neuen Chunks wiederholt werden sollen, wodurch eine Überlappung zwischen aufeinanderfolgenden Chunks entsteht. Diese Überlappung hilft dabei, die kontextuelle Kontinuität über Chunks hinweg aufrechtzuerhalten und verbessert so die Qualität und Genauigkeit der Einbettungen. Durch die systematische Segmentierung des Dokuments und die Generierung von Einbettungen für jeden Chunk verbessert die Funktion die Fähigkeit des Sprachmodells, relevante Informationen abzurufen, was zu präziseren und kontextuell angemessenen Antworten im RAG-Framework führt.

In [ ]:
def chunk_text(text, chunk_size, overlap):
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

In dieser Funktion setzen wir die Parameter für Blockgröße und Überlappung auf die folgenden Werte:

In [ ]:
chunk_size = 900
overlap = 300

Wir verarbeiten Textdateien, die Listen mit Biografien von Personen enthalten. Diese Einstellungen helfen dabei, die großen Textdaten effektiver zu verwalten. Die Blockgröße von 1000 stellt sicher, dass jedes Textsegment oder jeder Block bis zu 1000 Token enthält, was es dem Sprachmodell erleichtert, Einbettungen für jeden Teil zu verarbeiten und zu generieren. Die Überlappung von 200 Token bedeutet, dass jeder neue Block mit den letzten 200 Token des vorherigen Blocks beginnt. Diese Überlappung ist entscheidend, um die Kontinuität des Kontexts zwischen den Blöcken aufrechtzuerhalten, was besonders nützlich ist, wenn biografische Daten verarbeitet werden, bei denen sich Details oft über mehrere Blöcke erstrecken.


Der folgende Code liest Textinhalte aus einer Liste von URLs, verarbeitet diese Inhalte, indem er sie in kleinere, handhabbare Blöcke aufteilt, und speichert diese Blöcke als Dokumentobjekte. Zunächst wird eine leere Liste mit dem Namen „documents“ erstellt, um diese Dokumentobjekte aufzunehmen. Der Code durchläuft dann jede URL in der bereitgestellten Liste und gibt eine Meldung aus, die die aktuell verarbeitete URL angibt. Für jede URL ruft er den Inhalt mithilfe der Methode requests.get ab und prüft mit response.raise_for_status() auf HTTP-Fehler. Sobald der Inhalt erfolgreich abgerufen wurde, wird er mithilfe der Funktion chunk_text in kleinere Segmente aufgeteilt. Dabei werden die angegebene Blockgröße und Überlappung berücksichtigt, um die kontextuelle Kontinuität zwischen den Blöcken aufrechtzuerhalten. Diese Blöcke werden dann verwendet, um Dokumentobjekte zu erstellen, die jeweils einen Teil des Textes enthalten. Diese Dokumentobjekte werden anschließend an die Liste „documents“ angehängt, wodurch die großen Textdateien effektiv in kleinere, abrufbare Segmente organisiert werden, die für die weitere Verarbeitung geeignet sind, z. B. zum Generieren von Einbettungen für den LLM-Abruf in Retrieval-Augmented Generation (RAG).

In [ ]:
documents = []

for url in urls:
print(f"Lesen: {url}")
    response = requests.get(url)
    response.raise_for_status()  # Stellen Sie sicher, dass uns schlechte Antworten auffallen
    content = response.text
    chunks = chunk_text(content, chunk_size, overlap)
    for chunk in chunks:
        document = Document(page_content=chunk)
        documents.append(document)

Reading: https://data.heatonresearch.com/data/t81-559/bios/DD.txt
Reading: https://data.heatonresearch.com/data/t81-559/bios/FT.txt
Reading: https://data.heatonresearch.com/data/t81-559/bios/GS.txt
Reading: https://data.heatonresearch.com/data/t81-559/bios/NGS.txt
Reading: https://data.heatonresearch.com/data/t81-559/bios/TI.txt


Wir haben die Biografiedaten geladen und integrieren sie nun mit dem folgenden Code in die ChromaDB-Datenbank. ChromaDB, das ein Einbettungsmodell erfordert, ist standardmäßig auf „all-MiniLM-L6-v2“ eingestellt. Andere [model wrappers](https://docs.trychroma.com/integrations/openai) werden bereitgestellt. Dieses Modell wird von SentenceTransformerEmbeddings bereitgestellt, das hier für Einbettungsfunktionen instanziiert wird. Unter Verwendung von CharacterTextSplitter von langchain_text_splitters werden Dokumente für die Verarbeitung in handhabbare Teile aufgeteilt. Chroma, ein Vektorspeicher, lässt sich nahtlos mit diesen Komponenten verbinden und ermöglicht eine effiziente Speicherung und Abfrage von Einbettungen innerhalb der Datenbank.

In [ ]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain.vectorstores import Chroma

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(docs, embedding_function)

<ipython-input-6-11990a71100e>:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Wir sind dabei, eine RAG-Kette (Retrieval-Augmented Generation) mit dem folgenden Code zu konstruieren. Beginnend mit einem Retriever, der aus unserer zuvor konfigurierten ChromaDB-Datenbank initialisiert wurde, werden Dokumente zur Eingabevorbereitung in eine zusammengesetzte Zeichenfolge formatiert. Die Kette enthält einen Frage-Antwort-Fluss: Die formatierten Dokumente dienen neben einem Pass-Through-Fragehandler als Kontext. Unter Verwendung des vom Hub abgerufenen RAG-Modells geht die Kette zu einem Sprachmodell (llm) über. Schließlich werden die Ergebnisse mit StrOutputParser in ein Zeichenfolgenformat analysiert, wodurch der Prozess zur Generierung von Antworten basierend auf abgerufenen Dokumentkontexten abgeschlossen wird.

In [ ]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_prompt = hub.pull("rlm/rag-prompt")

def format_documents(documents):
    return "\n\n".join(doc.page_content for doc in documents)

retriever = db.as_retriever()

qa_chain = (
    {"context": retriever | format_documents, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:333: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langsmith/client.py:5434: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


Wir können jetzt die RAG-Kette aufrufen und sie zu einer der Personen abfragen, die in unseren Beispielbiografiedaten enthalten waren.

In [ ]:
qa_chain.invoke("What company does Elena Martinez work for?")

"I don't know."

## Die RAG-Kette auseinandernehmen

Mehrere Komponenten werden kombiniert, um externe Daten in eine Eingabeaufforderung für den RAG-Zugriff zu bringen. In diesem Abschnitt werden wir jede Komponente untersuchen und sehen, wie sie einzeln funktioniert. Wir beginnen damit, zu sehen, wie ChromaDB aufgefordert wird, und beobachten, wie es relevante Informationen aus dem größeren Satz von Dokumenten abruft. Diese kleinere Teilmenge ermöglicht es, die Daten in den Kontext einzufügen. Selbst wenn das gesamte Quellmaterial in den Kontextpuffer des LLM passen würde, würde RAG die Kosten senken und die Zugriffszeit verbessern, da viel weniger Daten an das LLM gesendet werden müssen.

In [ ]:
# frage es ab
query = "What company does Elena Martinez work for?"
docs = db.similarity_search(query)

# Ergebnisse drucken
print(docs)

[Document(page_content="rofessional life, Samantha is an avid rock climber and enjoys mentoring young women interested in STEM careers, aiming to inspire and cultivate a new generation of tech leaders.\n\nSamantha Clarke is a seasoned Project Manager at Digital Dynamics, a leading tech company known for its innovative solutions in digital marketing and AI-driven analytics. With over a decade of experience in the tech industry, Samantha has played a pivotal role in steering complex projects to success, enhancing the company's reputation for efficiency and cutting-edge technology. A graduate of MIT with a degree in Computer Science, she has a passion for integrating user-friendly technology with business needs. Samantha is particularly noted for her leadership in the development of the company's flagship product, the MarketMinder AI, which has revolutionized the way businesses understand consumer behavior. Outside"), Document(page_content="market reach. A graduate of MIT with a degree in

Wie Sie sehen, gibt ChromaDB mehrere Informationsbits zurück, die dem LLM den Kontext zur Beantwortung der Frage liefern. Keine dieser Informationen ist „allgemein bekannt“, sie sind nur in den generierten Biografiedaten vorhanden, die wir in ChromaDB geladen haben.

Wir werden uns auch die RAG-Eingabeaufforderungsvorlage ansehen. Wir verwenden die Standardeingabeaufforderungsvorlage von LangChain Hub. Der Text dieser Eingabeaufforderung ist hier. Sowohl die Frage als auch alle unterstützenden Informationen aus der Datenbank werden bereitgestellt.

In [ ]:
rag_prompt = hub.pull("rlm/rag-prompt")
rag_prompt.messages[0].prompt.template

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:333: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


"You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"

Das R in den RAG-Standards steht für „Retrieval“. Übergeben wir die Frage an das Retriever-Objekt, das ChromaDB abfragt, um herauszufinden, welche Dokumente am ehesten übereinstimmen. Sie werden einige Überlappungen und Duplikate sehen. Diese Überlappungen helfen dabei, Kontinuität über Blockgrenzen hinweg sicherzustellen.

In [ ]:
retriever.invoke("What company does Elena Martinez work for?")

[Document(page_content="rofessional life, Samantha is an avid rock climber and enjoys mentoring young women interested in STEM careers, aiming to inspire and cultivate a new generation of tech leaders.\n\nSamantha Clarke is a seasoned Project Manager at Digital Dynamics, a leading tech company known for its innovative solutions in digital marketing and AI-driven analytics. With over a decade of experience in the tech industry, Samantha has played a pivotal role in steering complex projects to success, enhancing the company's reputation for efficiency and cutting-edge technology. A graduate of MIT with a degree in Computer Science, she has a passion for integrating user-friendly technology with business needs. Samantha is particularly noted for her leadership in the development of the company's flagship product, the MarketMinder AI, which has revolutionized the way businesses understand consumer behavior. Outside"),
 Document(page_content="market reach. A graduate of MIT with a degree i

Diese Daten werden mit der Frage in der RAG-Eingabeaufforderung kombiniert, um sie an das LLM zu übermitteln.

## RAG über PDF-Dokumente

Wir werden nun untersuchen, wie man RAG mit einem PDF-Dokument verwendet. Wir werden ein PDF besprechen, das ich mit dem Buchersteller erstellt habe, den wir früher in dieser Klasse gesehen haben. Das Buch gehört zum Genre [steampunk](https://en.wikipedia.org/wiki/Steampunk) und trägt den Titel [steampunk](https://en.wikipedia.org/wiki/Steampunk). Wir verwenden denselben Code wie zuvor, außer dass ich das PDF in Text konvertiere, bevor ich Blöcke erstelle. Diese Technik wird Ihnen bei Aufgabe 6 sehr hilfreich sein.

In [ ]:
import pypdf
from io import BytesIO

MODEL = 'gpt-4o-mini'

llm = ChatOpenAI(
        model=MODEL,
        temperature=0.2,
        n=1
    )

urls = [
    "https://data.heatonresearch.com/data/t81-559/assignments/clockwork.pdf"
]

def extract_pdf_text(pdf_content):
    pdf_file = BytesIO(pdf_content)
    reader = pypdf.PdfReader(pdf_file)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

def chunk_text(text, chunk_size, overlap):
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

chunk_size = 900
overlap = 300

documents = []

for url in urls:
print(f"Lesen: {url}")
    response = requests.get(url)
    response.raise_for_status()  # Stellen Sie sicher, dass uns schlechte Antworten auffallen
    content = extract_pdf_text(response.content)  # Korrigiert, um Text mit pypdf zu extrahieren
    chunks = chunk_text(content, chunk_size, overlap)
    for chunk in chunks:
        document = Document(page_content=chunk)
        documents.append(document)

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(docs, embedding_function)
rag_prompt = hub.pull("rlm/rag-prompt")

def format_documents(documents):
    return "\n\n".join(doc.page_content for doc in documents)

retriever = db.as_retriever()

qa_chain = (
    {"context": retriever | format_documents, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

Reading: https://data.heatonresearch.com/data/t81-559/assignments/clockwork.pdf


/usr/local/lib/python3.10/dist-packages/langsmith/client.py:333: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Nachdem wir nun das PDF geladen haben, können wir es abfragen.

In [ ]:
qa_chain.invoke("Who is Eliza Hawthorne?")

'Eliza Hawthorne is a determined inventor and a central character in a narrative involving adventure and innovation in a steampunk setting. She seeks to challenge the status quo and uncover the truth behind forces manipulating technology in London. Eliza embodies the spirit of change and empowerment, ready to forge her own path amidst chaos.'

# 6.5: Einbetten von Datenbanken


In diesem Abschnitt untersuchen wir den Prozess der Ausführung von ChromaDB als Server, um den Zugriff auf RAG LLm zu erleichtern. Zunächst laden wir eine lokal gespeicherte Einbettungsdatenbank, die die Grundlage für die Datenverarbeitungsfunktionen unseres Servers bildet. Sobald die Datenbank erfolgreich geladen wurde, zeigen wir, wie sie neu geladen werden kann, um sicherzustellen, dass unser System aktuelle Informationen beibehalten und sich nahtlos von Störungen erholen kann. Schließlich konzentrieren wir uns auf den Zugriff auf die Datenbank über einen HTTP-Client, der es Benutzern ermöglicht, remote mit dem Server zu interagieren und Einbettungen nach Bedarf für effiziente und skalierbare Sprachmodelloperationen abzurufen und zu verwenden. In diesem Teil möchten wir eine umfassende Anleitung zum Einrichten und Verwalten von ChromaDB bereitstellen, damit ein robuster RAG LLm-Zugriff unterstützt wird.

Um die Fähigkeiten des Servers zu veranschaulichen, verwenden wir einen von mir erstellten Beispieldatensatz, der synthetische Daten von Mitarbeiterbiografien aus fünf fiktiven Unternehmen enthält. Dieser Datensatz soll zeigen, wie RAG bestimmte Informationen, die ein Basismodell nicht von sich aus besitzen würde, effektiv abrufen und nutzen kann. Auf diese Weise liefert er ein klares Beispiel dafür, wie RAG in einem realen Unternehmensumfeld angewendet werden kann.

Hier ist ein Beispiel für eine solche generierte Biografie:

> Elena Martinez ist eine erfahrene Robotikingenieurin bei FutureTech, einem führenden Innovator im Bereich künstliche Intelligenz und Robotik mit Sitz im Silicon Valley. Mit einem Master-Abschluss in Maschinenbau vom MIT und über einem Jahrzehnt Erfahrung war Elena maßgeblich an der Entwicklung autonomer Robotersysteme beteiligt, die die städtische Mobilität und Zugänglichkeit verbessern sollen. Zu ihren bahnbrechenden Arbeiten gehört die Entwicklung des ersten KI-gesteuerten Roboterassistenten, der nahtlos mit städtischen Umgebungen interagieren kann, um älteren und behinderten Menschen zu helfen. Als leidenschaftliche Verfechterin der Förderung von Frauen in MINT-Fächern leitet Elena auch das Outreach-Programm von FutureTech, das darauf abzielt, die nächste Generation weiblicher Ingenieure durch Workshops und Mentoring zu inspirieren. Ihre Beiträge haben FutureTech nicht nur zu neuen Höhen geführt, sondern auch neue Maßstäbe bei Roboteranwendungen für das Gemeinwohl gesetzt.


Diese Biografie zeigt die Art detaillierter, unternehmensspezifischer Informationen, die RAG abrufen und in seine Antworten integrieren kann, wodurch die Relevanz und Genauigkeit der generierten Inhalte verbessert wird. Durch die Integration dieser maßgeschneiderten Daten bereichert RAG nicht nur die Ergebnisse der LLMs, sondern stellt auch sicher, dass die Antworten auf den einzigartigen Kontext und die Bedürfnisse der Organisation abgestimmt sind.

Diese Beispieldaten werden unter den folgenden URLs gespeichert. Jede Datei enthält Personen aus einem der Unternehmen.

* https://data.heatonresearch.com/data/t81-559/bios/DD.txt
* https://data.heatonresearch.com/data/t81-559/bios/FT.txt
* https://data.heatonresearch.com/data/t81-559/bios/GS.txt
* https://data.heatonresearch.com/data/t81-559/bios/NGS.txt
* https://data.heatonresearch.com/data/t81-559/bios/TI.txt

Der folgende Code lädt diese Dokumente und speichert sie in einer ChromaDB-Datenbank. Diesen Code haben wir bereits zuvor gesehen.

## Laden Sie die ChromaDB-Datenbank



In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import TextLoader
from langchain import OpenAI, PromptTemplate
from langchain_openai import ChatOpenAI
from IPython.display import display_markdown
from langchain.indexes import VectorstoreIndexCreator
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.inmemory import InMemoryVectorStore
from langchain.schema import Document
import requests

urls = [
    "https://data.heatonresearch.com/data/t81-559/bios/DD.txt",
    "https://data.heatonresearch.com/data/t81-559/bios/FT.txt",
    "https://data.heatonresearch.com/data/t81-559/bios/GS.txt",
    "https://data.heatonresearch.com/data/t81-559/bios/NGS.txt",
    "https://data.heatonresearch.com/data/t81-559/bios/TI.txt"
]

def chunk_text(text, chunk_size, overlap):
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

chunk_size = 900
overlap = 300

documents = []

for url in urls:
print(f"Lesen: {url}")
    response = requests.get(url)
    response.raise_for_status()  # Stellen Sie sicher, dass uns schlechte Antworten auffallen
    content = response.text
    chunks = chunk_text(content, chunk_size, overlap)
    for chunk in chunks:
        document = Document(page_content=chunk)
        documents.append(document)

from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain.vectorstores import Chroma

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# db = Chroma.from_documents(Dokumente, Einbettungsfunktion)

Reading: https://data.heatonresearch.com/data/t81-559/bios/DD.txt
Reading: https://data.heatonresearch.com/data/t81-559/bios/FT.txt
Reading: https://data.heatonresearch.com/data/t81-559/bios/GS.txt
Reading: https://data.heatonresearch.com/data/t81-559/bios/NGS.txt
Reading: https://data.heatonresearch.com/data/t81-559/bios/TI.txt


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Abschließend laden wir aus diesen Dokumenten eine ChromaDB-Datenbank. Dabei geben wir allerdings zunächst ein Verzeichnis an, in dem diese Daten gespeichert werden.

In [ ]:
db = Chroma.from_documents(docs, embedding_function, persist_directory="/content/chroma_db")

Wir können sehen, dass dies ein reguläres Chroma-Objekt ist.

In [ ]:
type(db)

langchain_community.vectorstores.chroma.Chroma

Der folgende Code zeigt die Struktur der eingebetteten Datenbank, wie sie von ChromaDB gespeichert wird.

In [ ]:
!ls -al /content/chroma_db

total 5904
drwxr-xr-x 3 root root    4096 Jun 26 16:31 .
drwxr-xr-x 1 root root    4096 Jun 26 16:30 ..
-rw-r--r-- 1 root root 6029312 Jun 26 16:31 chroma.sqlite3
drwxr-xr-x 2 root root    4096 Jun 26 16:31 e7f2b890-35c6-4a12-b5ff-a12a08fe90c8


## Datenbank von der Festplatte neu laden

Wir können diese Daten nun in eine neue speicherbasierte ChromaDB neu laden. Diese Technik ist viel effizienter als das Neuladen der Daten mit der zuvor verwendeten Methode from_documents.

In [ ]:
db2 = Chroma(persist_directory="/content/chroma_db", embedding_function=embedding_function)


Der folgende Code, den wir zuvor besprochen haben, zeigt, dass die neu geladene Datenbank betriebsbereit ist.

In [ ]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

MODEL = 'gpt-4o-mini'

llm = ChatOpenAI(
        model=MODEL,
        temperature=0.2,
        n=1
    )

rag_prompt = hub.pull("rlm/rag-prompt")

def format_documents(documents):
    return "\n\n".join(doc.page_content for doc in documents)

retriever = db2.as_retriever()

qa_chain = (
    {"context": retriever | format_documents, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

Wir führen eine Abfrage durch, die auf die RAG-fähigen Daten zugreift.

In [ ]:
qa_chain.invoke("What company does Elena Martinez work for?")

'Elena Martinez works for Digital Dynamics.'

## Datenbank als Server ausführen

Mit dem folgenden Befehl können Sie einen ChromaDB-Server starten, der das zuvor erstellte Verzeichnis verwendet.

```
chroma run --host 127.0.0.1 --path /content/chroma_db &
```

Wir können jetzt einen Client erstellen, um mit diesem Datenbankserver zu kommunizieren.

In [ ]:
import chromadb
import langchain_community

client = chromadb.HttpClient(host='127.0.0.1', port=8000)
db3 = langchain_community.vectorstores.chroma.Chroma(
    client=client,embedding_function=embedding_function)

Der folgende Code, den wir zuvor besprochen haben, zeigt, dass die HTTP-Datenbank des Clients betriebsbereit ist.

In [ ]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

MODEL = 'gpt-4o-mini'

llm = ChatOpenAI(
        model=MODEL,
        temperature=0.2,
        n=1
    )

rag_prompt = hub.pull("rlm/rag-prompt")

def format_documents(documents):
    return "\n\n".join(doc.page_content for doc in documents)

retriever = db3.as_retriever()

qa_chain = (
    {"context": retriever | format_documents, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

Wir führen eine Abfrage durch, die auf die RAG-fähigen Daten zugreift.

In [ ]:
qa_chain.invoke("What company does Elena Martinez work for?")

'Samantha Clarke works for Digital Dynamics.'